# Knowledge Distillation For Fine-Tuning A GPT-3.5 Judge

There has been recent research that demonstrated GPT-4's ability to closely align to human judges when evaluating LLM generated texts (e.g., see [[1]](https://arxiv.org/abs/2306.05685), [[2]](https://arxiv.org/abs/2303.16634)). In this notebook, we demonstrate how to use the `llama_index` library to distill knowledge from GPT-4 to GPT-3.5 so that the smaller GPT-3.5 becomes closer to GPT-4 performance; and by proxy, closer to human judges.

To do so, we take the following steps:

1. Generate datasets: `train` and `test`
2. Perform knowledge distillation (using `train`)
3. Evaluate the distilled model  on `test`

## 0 Prompt Templates & Auxiliary Functions

In [ ]:
import os

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [ ]:
PROMPTS = {
    "QUESTION_GEN": (
        "You are a Teacher/ Professor. Your task is to setup "
        "a quiz/examination. Using the provided context, formulate "
        "a single question that captures an important fact from the "
        "context. Restrict the question to the context information provided."
    )
}

In [ ]:
import pandas as pd


# define jupyter display function
def display_eval_df(question, source, answer_a, answer_b, result) -> None:
    eval_df = pd.DataFrame(
        {
            "Question": question,
            "Source": source,
            "Model A": answer_a["model"],
            "Answer A": answer_a["text"],
            "Model B": answer_b["model"],
            "Answer B": answer_b["text"],
            "Score": result.score,
            "Judgement": result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        },
        subset=["Answer A", "Answer B"]
    )
    display(eval_df)

## 1 Generate datasets: `train` and `test`

We should not lose sight on the ultimate goal here, which is to build an LLM judge that closely matches to human judges when evaluating LLM-generated texts. The work we need to do in this step, therefore, is to build a set of generated texts that our LLM judges will judge. More specifically, we will follow the "pairwise comparison" evaluation design pattern, where one text generation is passed to an LLM judge that is subsequently prompted to assign a score between 0 and 1 (higher is better).

To generate a varied set of texts we'll use the following LLM text-generators:
1. HuggingFace: Llama2-7B (chat)
2. HuggingFace: Mistral-7B (instruct)
3. HuggingFace: Falcon-7B (instruct)

The generation task we ask of each of these models will be to generate an abstractive answer to question when provided relevant context (i.e., RAG).

### Using `DatasetGenerator` to build `train` and `test`

The specific procedure we will use here involves generating questions against a set of chunks of a given `Document`. With the `<question, chunk>` pairs in hand, (for which we can merely treat as a "simulated" retrieval), we pass this information to the three LLM generators and prompt them each to generate an answer.

Hang tight, we're almost there (sort of). Since we want to distill GPT-4 abilities for this task to GPT-3.5, we now need to generate GPT-4 judgements on the generated answers. To do this, we will pass the `<question, answer A, answer B>` (where `A` and `B` represent answers from any two of the LLM text-generators) as context to the GPT-4 judge and prompt it to decide the better answer of the two.

With all of that we can now build a `dataset` that looks like the one below.
| question | context-answer-A-answer-B | gpt-4-evaluation |
|----------|---------------------------|------------------|
| ...      | ...                       | ...              |

And finally, to get `train` and `test` we will simply randomly shuffle `dataset` and split it using a 70/30 ratio. (Phew!)

#### Generate Questions and LLM-Generated Answers

With all that out of the way, let's spring into action. First, we will download the reference pdf document and create the set of questions against it.

In [ ]:
# Download the pdf document — Uncomment the line of code below
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

In [ ]:
import random
from llama_index import SimpleDirectoryReader, ServiceContext

# load a document
documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

# Shuffle the documents
random.seed(42)
random.shuffle(documents)

In [ ]:
# generate questions against chunks
from llama_index.evaluation import DatasetGenerator
from llama_index.llms import OpenAI

# set context for llm provider
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

# instantiate a DatasetGenerator
dataset_generator = DatasetGenerator.from_documents(
    documents,
    question_gen_query=PROMPTS["QUESTION_GEN"],
    service_context=gpt_35_context,
    show_progress=True,
    num_questions_per_chunk=50,
)

In [ ]:
# use DatasetGenerator to create questions from nodes
questions = dataset_generator.generate_questions_from_nodes(num=25)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:01<00:00, 16.68it/s]


In [ ]:
# let's take a look at a few of these
for q in questions[:5]:
    print(q)

What was the author's first experience with programming on a microcomputer?
What language was regarded as the language of AI during the time period mentioned in the context?
Question: What was the author's motivation for considering a career in art?
What was the author's experience like studying at the Accademia?
What did the author learn about technology companies while working at Interleaf?


Now that we have the questions, the next step is to generate answers using the three LLM text-generators: Llama-2, Mistral, and Falcon. But, first we need to create a vector store for our documents and an associated retriever, which all of the LLM answer-generators will use.

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever

# Create vector index
the_index = VectorStoreIndex.from_documents(documents=documents)

# Create the retriver on this index
the_retriever = VectorIndexRetriever(
    index=the_index,
    node_ids=list(the_index.index_struct.nodes_dict.values()),
    similarity_top_k=2,
)

In [ ]:
from llama_index.query_engine.retriever_query_engine import (
    RetrieverQueryEngine,
)
from llama_index.llms import HuggingFaceInferenceAPI
from llama_index.llm_predictor import LLMPredictor


def create_query_engine(hf_name: str) -> RetrieverQueryEngine:
    """Create a RetrieverQueryEngine using the HuggingFaceInferenceAPI LLM"""
    if hf_name not in hf_llm_generators:
        raise KeyError("model not listed in hf_llm_generators")
    llm = HuggingFaceInferenceAPI(
        model_name=hf_llm_generators[hf_name],
        context_window=2048,  # to use refine
        token=HUGGING_FACE_TOKEN,
    )
    context = ServiceContext.from_defaults(llm_predictor=LLMPredictor(llm=llm))
    return RetrieverQueryEngine.from_args(
        retriever=the_retriever, service_context=context
    )

In [ ]:
# define our llm-generators
hf_llm_generators = {
    "mistral-7b-instruct": "mistralai/Mistral-7B-Instruct-v0.1",
    "llama2-7b-chat": "meta-llama/Llama-2-7b-chat-hf",
    "falcon-7b-instruct": "tiiuae/falcon-7b-instruct",
}

query_engines = {
    mdl: create_query_engine(mdl) for mdl in hf_llm_generators.keys()
}

/Users/nerdai/Library/Caches/pypoetry/virtualenvs/llama-index-e6cjsBOJ-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We're ready to now to produce the anaswers from the various LLMs.

In [ ]:
import tqdm

dataset = []
for q in tqdm.tqdm(questions):
    # randomly select two LLMs to generate answers to this q
    model_versus = random.sample(list(query_engines.items()), 2)

    # data for this q
    data_entry = {"question": q}
    responses = []
    source = None

    # generate answers
    for name, engine in model_versus:
        response = engine.query(q)
        response_struct = {}
        response_struct["model"] = name
        response_struct["text"] = str(response)
        if source is not None:
            assert source == response.source_nodes[0].node.text[:1000] + "..."
        else:
            source = response.source_nodes[0].node.text[:1000] + "..."
        responses.append(response_struct)

    data_entry["answers"] = responses
    data_entry["source"] = source
    data_entry["evaluation"] = None
    data_entry["fine_tuning_events"] = None
    dataset.append(data_entry)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [09:51<00:00, 32.84s/it]


In [ ]:
len(dataset)

18

In [ ]:
import json

# save these generations for future use
with open("qa_dataset.jsonl", "w") as outfile:
    for entry in dataset:
        print(json.dumps(entry), file=outfile)

#### Generate GPT-4 Evaluations

In [ ]:
# for loading the jsonl file
# import json

# with open("qa_dataset.jsonl") as f:
#     dataset = [json.loads(line) for line in f]

#### Build Custom Evaluator

In [ ]:
DEFAULT_SYSTEM_TEMPLATE = (
    "Please act as an impartial judge and evaluate the quality of the responses provided by two "
    "AI question-answering assistants to the user question along with the retrieved context which "
    "was provided to both assistants are displayed below. You should choose the assistant that "
    "follows the user’s instructions and answers the user’s question better using the provided "
    "context. Your evaluation "
    "should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, "
    "and level of detail of their responses. Begin your evaluation by comparing the two "
    "responses and provide a short explanation. Avoid any position biases and ensure that the "
    "order in which the responses were presented does not influence your decision. Do not allow "
    "the length of the responses to influence your evaluation. Do not favor certain names of "
    "the assistants. Be as objective as possible. After providing your explanation, output your "
    "final verdict by strictly following this format: '[[A]]' if assistant A is better, '[[B]]' "
    "if assistant B is better, and '[[C]]' for a tie.\n"
)

DEFAULT_USER_TEMPLATE = (
    "[User Question]\n"
    "{question}"
    "\n\n"
    "[The Start Retrieved Source]\n"
    "{source}\n"
    "[The End of Retrieved Source]"
    "\n\n"
    "[The Start of Assistant A’s Answer]\n"
    "{answer_a}\n"
    "[The End of Assistant A’s Answer]"
    "\n\n"
    "[The Start of Assistant B’s Answer]\n"
    "{answer_b}\n"
    "[The End of Assistant B’s Answer]"
)

In [ ]:
from llama_index.evaluation.base import EvaluationResult
from llama_index.prompts import (
    BasePromptTemplate,
    ChatMessage,
    ChatPromptTemplate,
    MessageRole,
    PromptTemplate,
)
from llama_index.prompts.mixin import (
    PromptDictType,
    PromptMixin,
    PromptMixinType,
)
from typing import Tuple

DEFAULT_EVAL_TEMPLATE = ChatPromptTemplate(
    message_templates=[
        ChatMessage(role=MessageRole.SYSTEM, content=DEFAULT_SYSTEM_TEMPLATE),
        ChatMessage(role=MessageRole.USER, content=DEFAULT_USER_TEMPLATE),
    ]
)


def parse_eval_result(
    query: str,
    response: str,
) -> EvaluationResult:
    """Take an Evaluation Result and parse response to assign a score.
    - 1.0 if Answer A is better than Answer B
    - 0.0 if Answer B is better than Answer A
    - 0.5 if tie
    """

    score = None
    if "[[A]]" in response:
        score = 1.0
    elif "[[B]]" in response:
        score = 0.0
    elif "[[C]]" in response:
        score = 0.5
    else:
        raise ValueError("Unable to parse response")

    return EvaluationResult(
        query=query,
        response=response,
        score=score,
        feedback=response,
    )


def resolve_results(
    eval_result: EvaluationResult,
    flipped_eval_result: EvaluationResult,
) -> Tuple[EvaluationResult, str]:
    """Resolve eval results from evaluation + flipped evaluation."""
    votes_a = eval_result.score + (1 - flipped_eval_result.score)
    votes_b = (1 - eval_result.score) + flipped_eval_result.score
    assert votes_a + votes_b == 2

    a_voters = [(eval_result, "original")] * (eval_result.score == 1.0) + [
        (flipped_eval_result, "flipped")
    ] * (flipped_eval_result.score == 0.0)

    b_voters = [(eval_result, "original")] * (eval_result.score == 0.0) + [
        (flipped_eval_result, "flipped")
    ] * (flipped_eval_result.score == 1.0)

    if votes_a > votes_b:
        return a_voters[0]
    elif votes_b > votes_a:
        return b_voters[0]
    else:
        if eval_result.score == 0.5:  # voted tie twice
            return (eval_result, "original")
        else:
            return (
                EvaluationResult(
                    query=eval_result.query,
                    response="",
                    passing=None,
                    score=0.5,
                    feedback="Inconclusive.",
                ),
                "",
            )

In [ ]:
# instantiate the gpt-4 judge
from llama_index.llms import OpenAI
from llama_index import ServiceContext
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])
gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(temperature=0, model="gpt-4"),
    callback_manager=callback_manager,
)

flipped_finetuning_handler = OpenAIFineTuningHandler()
flipped_callback_manager = CallbackManager([flipped_finetuning_handler])
flipped_gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(temperature=0, model="gpt-4"),
    callback_manager=flipped_callback_manager,
)

# gpt4_judge = PairwiseComparisonEvaluator(service_context=gpt_4_context)

In [ ]:
for data_entry in tqdm.tqdm(dataset):
    result = await gpt_4_context.llm_predictor.apredict(
        prompt=DEFAULT_EVAL_TEMPLATE,
        question=data_entry["question"],
        source=data_entry["source"],
        answer_a=data_entry["answers"][0]["text"],
        answer_b=data_entry["answers"][1]["text"],
    )
    eval_result = parse_eval_result(data_entry["question"], result)

    # flip A and B for addressing position bias
    result = await flipped_gpt_4_context.llm_predictor.apredict(
        prompt=DEFAULT_EVAL_TEMPLATE,
        question=data_entry["question"],
        source=data_entry["source"],
        answer_a=data_entry["answers"][1]["text"],
        answer_b=data_entry["answers"][0]["text"],
    )
    flipped_eval_result = parse_eval_result(data_entry["question"], result)

    # merge result
    final_eval_result, judgement_source = resolve_results(
        eval_result,
        flipped_eval_result,
    )

    # save final result
    judgement = {}
    judgement["llm"] = "gpt_4"
    judgement["score"] = final_eval_result.score
    judgement["text"] = final_eval_result.response
    judgement["source"] = judgement_source
    data_entry["evaluations"] = [judgement]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [07:49<00:00, 26.07s/it]


In [ ]:
# save these generations for future use
import json

with open("qa_dataset.jsonl", "w") as outfile:
    for entry in dataset:
        print(json.dumps(entry), file=outfile)

In [ ]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")
flipped_finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 19 examples to finetuning_events.jsonl
Wrote 19 examples to finetuning_events.jsonl


In [ ]:
# Get the fine_tuning_examples master dataset
with open("finetuning_events.jsonl") as f:
    finetuning_events = [json.loads(line) for line in f]

with open("finetuning_events.jsonl") as f:
    flipped_finetuning_events = [json.loads(line) for line in f]

assert len(finetuning_events) == len(flipped_finetuning_events)

In [ ]:
final_finetuning_events = []
for ix, data_entry in enumerate(dataset):
    if data_entry["evaluations"][0]["source"] == "original":
        final_finetuning_events += [finetuning_events[ix]]
    elif data_entry["evaluations"][0]["source"] == "flipped":
        final_finetuning_events += [flipped_finetuning_events[ix]]
    else:
        continue

In [ ]:
with open("final_finetuning_events.jsonl", "w") as outfile:
    for entry in final_finetuning_events:
        print(json.dumps(entry), file=outfile)

In [ ]:
len(final_finetuning_events)

17

Let's just see how one of these looks like.

In [ ]:
# let's see the last one
display_eval_df(
    question=data_entry["question"],
    source=data_entry["source"],
    answer_a=data_entry["answers"][0],
    answer_b=data_entry["answers"][1],
    result=final_eval_result,
)

,Question,Source,Model A,Answer A,Model B,Answer B,Score,Judgement
0,What is an example of a concept that would likely be known to a sufficiently advanced alien civilization?,"[19] One way to get more precise about the concept of invented vs discovered is to talk about space aliens. Any sufficiently advanced alien civilization would certainly know about the Pythagorean theorem, for example. I believe, though with less certainty, that they would also know about the Lisp in McCarthy's 1960 paper. But if so there's no reason to suppose that this is the limit of the language that might be known to them. Presumably aliens need numbers and errors and I/O too. So it seems likely there exists at least one path out of McCarthy's Lisp along which discoveredness is preserved. Thanks to Trevor Blackwell, John Collison, Patrick Collison, Daniel Gackle, Ralph Hazell, Jessica Livingston, Robert Morris, and Harj Taggar for reading drafts of this....",llama2-7b-chat,"The Pythagorean theorem is an example of a concept that would likely be known to a sufficiently advanced alien civilization. Explanation: The Pythagorean theorem is a fundamental concept in mathematics that has been widely studied and applied across various fields, including physics, engineering, and computer science. It is a simple yet powerful idea that can be easily understood and applied, making it a likely candidate for a concept that would be known to a sufficiently advanced alien civilization. Additionally, the author of the passage suggests that any sufficiently advanced alien civilization would likely have a similar understanding of numbers, errors, and I/O, which are fundamental components of computer science and programming. Therefore, it is likely that a sufficiently advanced alien civilization would be familiar with concepts like Lisp, which is a programming language that is widely used in computer science and artificial intelligence.",mistral-7b-instruct,An example of a concept that would likely be known to a sufficiently advanced alien civilization is the Pythagorean theorem.,1.000000,"Assistant A provides a more detailed and comprehensive response. While both assistants correctly identify the Pythagorean theorem as an example of a concept that would likely be known to a sufficiently advanced alien civilization, Assistant A goes further to explain why this is the case. Assistant A also mentions other concepts such as numbers, errors, and I/O, and the Lisp programming language, which are suggested in the provided context as likely known to an advanced alien civilization. Therefore, Assistant A's response is more helpful, accurate, and detailed. Final Verdict: [[A]]"


## 2 Perform knowledge distillation

Okay, it's now time to distill some knowledge from GPT-4 to GPT-3.5 To do this, we will make use of `OpenAIFinetuneEngine` class of `llama_index`.

In [ ]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "final_finetuning_events.jsonl",
)

In [ ]:
finetune_engine.finetune()

Num examples: 17
First example:
{'role': 'system', 'content': "Please act as an impartial judge and evaluate the quality of the responses provided by two AI question-answering assistants to the user question along with the retrieved context which was provided to both assistants are displayed below. You should choose the assistant that follows the user’s instructions and answers the user’s question better using the provided context. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by 

We can check the status of our current job as follows:

In [ ]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-Sov8D6e31JITVoN8oDi9mB21 at 0x1744e1440> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-Sov8D6e31JITVoN8oDi9mB21",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698706048,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-1ZDAvajC6v2ZtAP9hLEIsXRz",
  "result_files": [],
  "status": "queued",
  "validation_file": null,
  "training_file": "file-GEiTBygEV5QFEq8Vgf4eniUM",
  "hyperparameters": {
    "n_epochs": 5
  },
  "trained_tokens": null,
  "error": null
}

## 3 Evaluation

In [ ]:
ft_llm = finetune_engine.get_finetuned_model()

In [ ]:
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
)

# a non-fine-tuned judge
gpt_3p5_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo")
)

In [ ]:
# predicting on training set for now just to get rest of pipeline established
for data_entry in tqdm.tqdm(dataset):
    question = data_entry["question"]

    result = await ft_context.llm_predictor.apredict(
        prompt=DEFAULT_EVAL_TEMPLATE,
        question=data_entry["question"],
        source=data_entry["source"],
        answer_a=data_entry["answers"][0]["text"],
        answer_b=data_entry["answers"][1]["text"],
    )
    eval_result = parse_eval_result(data_entry["question"], result)

    # flip A and B for addressing position bias
    flipped_result = await ft_context.llm_predictor.apredict(
        prompt=DEFAULT_EVAL_TEMPLATE,
        question=data_entry["question"],
        source=data_entry["source"],
        answer_a=data_entry["answers"][1]["text"],
        answer_b=data_entry["answers"][0]["text"],
    )
    flipped_eval_result = parse_eval_result(
        data_entry["question"], flipped_result
    )

    # merge result
    final_eval_result, judgement_source = resolve_results(
        eval_result,
        flipped_eval_result,
    )

    # save final result
    judgement = {}
    judgement["llm"] = "ft_gpt_3p5"
    judgement["score"] = final_eval_result.score
    judgement["text"] = final_eval_result.response
    judgement["source"] = judgement_source
    data_entry["evaluations"] += [judgement]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [01:00<00:00,  3.37s/it]


In [ ]:
# predicting on training set for now just to get rest of pipeline established
for data_entry in tqdm.tqdm(dataset):
    question = data_entry["question"]

    result = await gpt_3p5_context.llm_predictor.apredict(
        prompt=DEFAULT_EVAL_TEMPLATE,
        question=data_entry["question"],
        source=data_entry["source"],
        answer_a=data_entry["answers"][0]["text"],
        answer_b=data_entry["answers"][1]["text"],
    )
    try:
        eval_result = parse_eval_result(data_entry["question"], result)
    except:
        eval_result = EvaluationResult(
            query=eval_result.query,
            response="",
            passing=None,
            score=0.5,
            feedback="Didn't follow output criteria.",
        )

    # flip A and B for addressing position bias
    flipped_result = await gpt_3p5_context.llm_predictor.apredict(
        prompt=DEFAULT_EVAL_TEMPLATE,
        question=data_entry["question"],
        source=data_entry["source"],
        answer_a=data_entry["answers"][1]["text"],
        answer_b=data_entry["answers"][0]["text"],
    )
    flipped_eval_result = parse_eval_result(
        data_entry["question"], flipped_result
    )

    # merge result
    final_eval_result, judgement_source = resolve_results(
        eval_result,
        flipped_eval_result,
    )

    # save final result
    judgement = {}
    judgement["llm"] = "gpt_3p5"
    judgement["score"] = final_eval_result.score
    judgement["text"] = final_eval_result.response
    judgement["source"] = judgement_source
    data_entry["evaluations"] += [judgement]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [01:45<00:00,  5.86s/it]


In [ ]:
# store for future analyses
with open("qa_dataset_complete.jsonl", "w") as outfile:
    for entry in dataset:
        print(json.dumps(entry), file=outfile)

Let's now compute the agreement rates between the llm-judges.

In [ ]:
import numpy as np

scores = {"gpt_4": [], "gpt_3p5": [], "ft_gpt_3p5": []}
for ix, d in enumerate(dataset):
    responses = [
        el["text"] for el in d["evaluations"]
    ]  # remove any inconclusive results from any of the judges
    if any(t == "" for t in responses):
        print(f"{ix}: inconclusive")
        continue
    for e in d["evaluations"]:
        scores[e["llm"]].append(e["score"])

7: inconclusive
11: inconclusive
12: inconclusive
14: inconclusive
16: inconclusive


In [ ]:
# agreement rates
agreement_ft = sum(
    x == y for x, y in zip(scores["gpt_4"], scores["ft_gpt_3p5"])
)
agreement_no_ft = sum(
    x == y for x, y in zip(scores["gpt_4"], scores["gpt_3p5"])
)

# final report
print(
    f"GPT-3.5 w/ fine-tuning\n----------------\nNumber of agreements with GPT-4: {agreement_ft}\nagreement rate: {agreement_ft/len(scores['gpt_4'])}"
)
print("\n")
print(
    f"GPT-3.5 w/out fine-tuning\n----------------\nNumber of agreements with GPT-4: {agreement_no_ft}\nagreement rate: {agreement_no_ft/len(scores['gpt_4'])}"
)

GPT-3.5 w/ fine-tuning
----------------
Number of agreements with GPT-4: 8
agreement rate: 0.6153846153846154


GPT-3.5 w/out fine-tuning
----------------
Number of agreements with GPT-4: 7
agreement rate: 0.5384615384615384


So, we can see that with fine-tuning our GPT-3.5 model gets close to GPT-4 and thus, by proxy, closer to human judgement.